In [1]:
extract = []
file = open("data.txt", "r")
for i in file:
    extract.append(i)
    

In [3]:
print(f"Size of list before removal of duplicates: {len(extract)}")
res = [*set(extract)]
print(f"Size of list after removal of duplicates: {len(res)}")
print(f"Number of duplicates removed: {len(extract) - len(res)}")

Size of list before removal of duplicates: 1678305
Size of list after removal of duplicates: 1660970
Number of duplicates removed: 17335


-1 => negative <br>
0 => neutral <br>
1 => positive <br>

In [8]:
new = []
for i in res:
    if "negative" in i:
        new.append(i.replace("negative", "-1"))
    elif "positive" in i:
        new.append(i.replace("positive", "1"))
    elif "neutral" in i:
        new.append(i.replace("neutral", "0"))

In [9]:
file = open("cleaned_data.txt", "w+")
for i in new:
    file.write(i)
file.close()

In [11]:
!rm data.txt
!mv cleaned_data.txt data.txt

rm: cannot remove 'data.txt': No such file or directory


In [ ]:
#trying to commit